In [1]:
import numpy as np
import scipy
import sklearn
import scipy.stats as stats
import pandas as pd
import fc_subtypes_functions as func


from sklearn import datasets
from statsmodels.stats import multitest
from copy import deepcopy




In [2]:
# Load Weights and intitialze variables

networks = [ 'CER','LIM','MOT','VIS','DMN','FP','VATTSAL'] 
labels = ["1","2","3","4","5","6","7"]

%store -r w_2
weights = w_2
num_subtypes = 3

%store -r selection_demog
selected_subjects = selection_demog.index

networks_weights_dict = dict()
contingency_table_dict = dict()

demo = pd.read_csv("adni_cimaq_info_niak.csv").rename(columns = {'Unnamed: 0': "subjectID"})
demo = demo[list(demo.columns[0:11])]


In [3]:

for net in labels:
    network = int(net)-1
    label_network = networks[int(net)-1]
    
    #Create a dataframe for each network
    pd_weights = pd.DataFrame(columns = ["subjectID"])
    
    pd_weights["subjectID"] = selected_subjects
    
    for subtype_ in range(num_subtypes):
        subtype_curr = "Subtype"+str(subtype_+1)+"_"+label_network
        print(subtype_curr,)
        weight = weights[:,network*num_subtypes+subtype_]
        pd_weights[subtype_curr] = weight
        
    
    print(" ")
    #create a dataframe for each network 
    networks_weights_dict.update({label_network:pd_weights})
    networks_weights_dict[label_network] = func.max_subtype(networks_weights_dict[label_network])
    
    #merge with demographic information 
    networks_weights_dict[label_network] = networks_weights_dict[label_network].merge(demo, how="left" ,on="subjectID")
    
    #create a contingency table for each network 
    contingency_table = func.create_ct(networks_weights_dict[label_network],num_subtypes)
    contingency_table_dict.update({label_network:contingency_table})

Subtype1_CER
Subtype2_CER
Subtype3_CER
 
Subtype1_LIM
Subtype2_LIM
Subtype3_LIM
 
Subtype1_MOT
Subtype2_MOT
Subtype3_MOT
 
Subtype1_VIS
Subtype2_VIS
Subtype3_VIS
 
Subtype1_DMN
Subtype2_DMN
Subtype3_DMN
 
Subtype1_FP
Subtype2_FP
Subtype3_FP
 
Subtype1_VATTSAL
Subtype2_VATTSAL
Subtype3_VATTSAL
 


In [4]:
network_interest = "LIM" 

In [5]:
networks_weights_dict[network_interest].head()

,subjectID,Subtype1_LIM,Subtype2_LIM,Subtype3_LIM,max_weight,max_subtype,age,Female,CN,SCI,MCI,AD,QC,FD_scrubbed,adni2,cimaq
0,s0107,-0.255723,0.094996,-0.110578,-0.255723,Subtype1_LIM,67.0,1,0,0,1,0,1,0.153941,1,0
1,s0186,-0.034018,-0.032633,0.014972,-0.034018,Subtype1_LIM,87.0,1,1,0,0,0,1,0.173473,1,0
2,s0295,-0.095864,-0.050840,-0.234918,-0.234918,Subtype3_LIM,90.0,0,1,0,0,0,1,0.246640,1,0
3,s0685,-0.087883,0.010528,0.054977,-0.087883,Subtype1_LIM,94.0,1,1,0,0,0,1,0.209432,1,0
4,s0729,0.082648,-0.044959,0.150956,0.150956,Subtype3_LIM,70.0,1,0,0,0,1,1,0.191809,1,0


In [6]:
contingency_table_dict[network_interest]

,class,Subtype1_LIM,Subtype2_LIM,Subtype3_LIM
0,CN,30,13,15
1,ADMCI,54,18,43


# Discrete Associations


In [7]:
discrete_associations = pd.DataFrame(columns = ["network","pchi","qfdr","chiV"])
pchi_arr = []
qfdr_arr = []
chiV_arr = []

for net in networks:


    ct_table = deepcopy(contingency_table_dict[net])
    del ct_table["class"]
    pchi = stats.chi2_contingency(ct_table.values, correction=False, lambda_=None)
    pchi_arr.append(pchi[1])
    qfdr_arr.append(multitest.multipletests(pchi[1],0.05,'fdr_bh')[1][0])
    chiV_arr.append(func.cramers_corrected_stat(ct_table.values,pchi[0]))
    
    
discrete_associations["network"] =  networks  
discrete_associations["pchi"] =  pchi_arr  
discrete_associations["qfdr"] =  qfdr_arr  
discrete_associations["chiV"] =  chiV_arr  


discrete_associations

,network,pchi,qfdr,chiV
0,CER,0.110027,0.110027,0.118187
1,LIM,0.260174,0.260174,0.062933
2,MOT,0.983047,0.983047,0.000000
3,VIS,0.277143,0.277143,0.056796
4,DMN,0.088810,0.088810,0.128293
5,FP,0.930632,0.930632,0.000000
6,VATTSAL,0.069302,0.069302,0.139080



# Continuous Associations


In [8]:

for net in networks:
    for subtype_x in networks_weights_dict[net].columns[1:num_subtypes+1]:
        pd_network = networks_weights_dict[net]
        X = pd_network[["CN"]].values
        y = pd_network[[subtype_x]].values
        scipy.io.savemat('./glm_mat/'+subtype_x+'.mat', dict(x=X, y=y))  
        


In [13]:
glm_output = pd.read_csv("glm_mat/glm_output.csv")
glm_output

,,ttest,pce
0,Subtype1_CER,0.474843,0.635353
1,Subtype2_CER,0.950176,0.343029
2,Subtype3_CER,-0.319591,0.749571
3,Subtype1_LIM,-0.843605,0.399775
4,Subtype2_LIM,-0.253731,0.799932
5,Subtype3_LIM,-0.328037,0.743185
6,Subtype1_MOT,-0.359997,0.719183
7,Subtype2_MOT,0.064087,0.948957
8,Subtype3_MOT,0.851135,0.395587
9,Subtype1_VIS,-0.308678,0.757849
